In [1]:
import pandas as pd
import math
import numpy as np
from sklearn import metrics

In [2]:
def find_cutpoint(read_record_path,read_pred_path):
    record_cutpoint = []
    record_file_name = read_record_path + "/start_plant_list.csv"
    record_fault_name = read_record_path + "/start_fault_list.csv"
    record_vali_name = read_pred_path + "/pred_start/record_vali.csv"
    record_file =  np.genfromtxt(record_file_name, delimiter=',', dtype=np.int32)
    record_fault = np.genfromtxt(record_fault_name, delimiter=',', dtype=np.int32)
    record_vali = np.genfromtxt(record_vali_name,delimiter=',', dtype=np.int32)
    for n_index in range(0, 2):
        if( record_vali[n_index] == 1):
            ##pred_trainset_name = read_nosample_path + "/trainset" + str(record_file[n_index]) + "_fault"+str(record_fault[n_index]) + ".csv"
            pred_vali_name = read_pred_path + "/pred_start/validate/start_vali_5conv" + str(record_file[n_index]) + "_fault"+str(record_fault[n_index]) + ".csv"
            ##pred_trainset = pd.read_csv(pred_trainset_name)
            pred_vali    = pd.read_csv(pred_vali_name)
            semi_pred_vali = []
            semi_pred_vali_label = []
            
            ##simlulating the testing record
            for i in range(math.floor(pred_vali.shape[0]/6)-1):
                best_prob = 0
                true_label = 0
                for j in range(i*6,(i+1)*6):
                    if pred_vali.one[j] > best_prob:
                        best_prob = pred_vali.one[j]
                        true_label = pred_vali.true[j]
                semi_pred_vali.append(best_prob)
                semi_pred_vali_label.append(true_label)
            semi_pred_vali = np.array(semi_pred_vali)
            semi_pred_vali_label = np.array(semi_pred_vali_label)

            fpr, tpr, thresholds = metrics.roc_curve(pred_vali.true, pred_vali.one)
            cut_vali = pd.DataFrame({"pred":semi_pred_vali, "true": semi_pred_vali_label, "ans":[0]*semi_pred_vali.shape[0]})
            score_eval = []
            for prob in thresholds:
                cut_vali.ans = 0
                cut_vali.loc[cut_vali.pred>prob,"ans"] = 1
                cut_vali_one = cut_vali.loc[cut_vali.ans == 1,:]
                cut_vali_zero = cut_vali.loc[cut_vali.ans == 0,:]
                
                TP = (cut_vali_one.true == cut_vali_one.ans).sum()
                FP = (cut_vali_one.true != cut_vali_one.ans).sum()
                FN = (cut_vali_zero.true == cut_vali_zero.ans).sum()
                score_eval.append(TP*10-(FP+FN)*0.1)
            ##find the corresponding cutpoint
            cutpoint = thresholds[np.argmax(score_eval)]
            if(cutpoint < 0.5):
                cutpoint = 0.5
            record_cutpoint.append(cutpoint)
            print("index: %s  total:%s  cutpoint:%s"%(n_index,len(record_file),cutpoint))
        else:
            record_cutpoint.append(0)
    f_name = read_pred_path + "/pred_start/record_cutpoint.csv"
    f = open( f_name,"w")
    for index in range( (len(record_cutpoint)-1) ):
        f.write(str(record_cutpoint[index])+",")
    f.write(str(record_cutpoint[-1]))
    f.close()

In [3]:
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value

    """
    fpr, tpr, threshold = metrics.roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]
    ##https://stackoverflow.com/questions/28719067/roc-curve-and-cut-off-point-python
    return list(roc_t['threshold']) 

In [4]:
def find_cutpoint2(read_record_path,read_pred_path):
    record_cutpoint = []
    record_file_name = read_record_path + "/start_plant_list.csv"
    record_fault_name = read_record_path + "/start_fault_list.csv"
    record_vali_name = read_pred_path + "/pred_start/record_vali.csv"
    record_file =  np.genfromtxt(record_file_name, delimiter=',', dtype=np.int32)
    record_fault = np.genfromtxt(record_fault_name, delimiter=',', dtype=np.int32)
    record_vali = np.genfromtxt(record_vali_name,delimiter=',', dtype=np.int32)
    for n_index in range(len(record_file)):
        if( record_vali[n_index] == 1):
            pred_vali_name = read_pred_path + "/pred_start/validate/start_vali_5conv" + str(record_file[n_index]) + "_fault"+str(record_fault[n_index]) + ".csv"
            pred_vali    = pd.read_csv(pred_vali_name)

            cutpoint = Find_Optimal_Cutoff(pred_vali.true, pred_vali.one)[0]
            if(cutpoint < 0.5):
                cutpoint = 0.5
            record_cutpoint.append(cutpoint)
            print("index: %s  total:%s  cutpoint:%s"%(n_index,len(record_file),cutpoint))
        else:
            record_cutpoint.append(0)
    f_name = read_pred_path + "/pred_start/record_cutpoint.csv"
    f = open( f_name,"w")
    for index in range( (len(record_cutpoint)-1) ):
        f.write(str(record_cutpoint[index])+",")
    f.write(str(record_cutpoint[-1]))
    f.close()

In [5]:
read_record_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/PHM_nosample/file"
read_pred_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/20170831DEMO/file"

find_cutpoint(read_record_path, read_pred_path)

index: 0  total:135  cutpoint:0.819825649261
index: 1  total:135  cutpoint:0.894259333611
